**Struct Package**

Using binary files can be extremely powerful. In Python we can store huge amounts of data for fast retrieval if our data consists of a sequence of records all consisting of the same structure (hence same number of bytes per record).

Here is a link to info about the struct package from the python reference manual.

https://docs.python.org/3/library/struct.html

**Sample Application**

To motivate the use of structs, consider the following situation.

The Fannie-Mae single family mortgage performance dataset consists of a .csv file for loand originating in every quarter since the year 2000.

Each quarter there are between 250,000 - 1,500,000 loans originating in that quarter.

In the csv file, for each loan there is 

- a first line giving information available at the time the loan was originated
- a series of lines, one for each month about payments made on the loan and additional information about
    - whether default occured
    - whether refinancing occured
    - whethe the loan was prepayed

The entire dataset take up about 650Gb of disk space. Each file can be a gigabyte or 2 in size.

These are all text files. Reading one into Python for analysis is very slow.

We'd like to be able to get the information for a given loan quickly. 

So we do the following.

We determine the size of each column/field needed to store the information in that column in bytes.

Each row requires 480 bytes. The information about the fields is stored in a file "variables_in_binary_files.csv".

For each .csv file, we create a binary file with the data for each row in the csv file.

We also store in a separate file e.g. "2002Q3LIDinfo.csv" for each quarter the range of rows in which each loan ID appears.

**Sample calculation**

Here is an example of how we can use these file to do quick data analysis. We want to get information about a loan in the first line of a given quarter. We illustrate with 2003 3rd quarter. 

In [58]:
import pandas as pd
df=pd.read_csv("2002Q3LIDinfo.csv",header=None)
df.columns=["LID","start","nrows"]

index=41894

LID=df.LID[index]
start=df.start[index]
nrows=df.nrows[index]

print(LID)
print(start)
print(nrows)

150306578935
1846680
59


In [59]:
with open("2002Q3.bin","rb") as fin:
    fin.seek(start*480)
    row=fin.read(480)
    print(row)

b'150306578935082002B                                               Bank Of America, N.A.                                             Bank Of America, N.A. 6.875 6.875 146000.00      0.00360052002072002  235835806203285 85 235728737NPSF1PNJ1210008212.00FRMN       00                                                N                                                                                                                                                   1N          N N N   '


**Extract the data for a row**

And now we want to extract information in a given row. For example, suppose we want the current interest rate.

In [61]:
dv=pd.read_csv("variables_in_binary_files.csv")
dv

,varname,nchars,field_start,field_end
0,LOAN_ID,12,0,12
1,MONTHLY_REPORTING_PERIOD,6,12,18
2,CHANNEL,1,18,19
3,SELLER_NAME,68,19,87
4,SERVICER_NAME,66,87,153
...,...,...,...,...
63,BORROWER_ASSISTANCE_PLAN,1,475,476
64,HLTV,1,476,477
65,REPURCHASE_MAKE_WHOLE_PROC_FLAG_I,1,477,478
66,ALTERNATIVE_DELINQUENCY_RESOLUTION,1,478,479


In [63]:
# print(dv.columns)
# print(dv.varname[6])
# print(dv.nchars[6])
# print(dv.field_start[6])

row[dv.field_start[6]:dv.field_end[6]]

6.875

Note that the result is a byte string. 

We can do this to get a floating point value in some cases.

In [64]:
float(row[dv.field_start[6]:dv.field_end[6]])

6.875

**The structs package**

To write fields to a file we converting certain python objects to byte strings of fixed length (number of bytes).

This is where the struct package becomes useful.

**Another application**

To motivate this, consider the following application, which is now quite common in microbiology. In a single cell RNA seq experiment (scRNA-seq), for a huge number of cells (e.g. 100,000) and for a large number of MRNA's (e.g. 20,000) the count of MRN's expressed for each cell is determined. The result could, in principle, be stored in matrix that is 20,000 x 100,000, so around 2 billion entries.

In fact there is typically a great deal of sparsity. Each cell might only typically express about 5% of the MRN's so roughly 100 million entries are nonzero in the matrix. 
Rather than store all of those zeros, a typical alternative is to use a sparse matrix representation, creating a list of (row,column,value) 3-tuples one for each nonzero entry. This still leads to a very large text file which will take a long time to read in.

A standard file format for storing a sparse matrix is called MatrixMarket format (.mtx is a common file extension for such a file.

https://math.nist.gov/MatrixMarket/formats.html

Here is a very small example (datas.mtx). In this example, the values 

%% MatrixMarket matrix coordinate integer general
%
% each data entry after structure line gives row, column, value (real) 
%
% the following structure line gives #rows #columns #nonzero entries
5 8 10
1 2 18
1 3 7
1 4 6
2 3 2
2 4 1
2 5 1
4 1 1
4 4 2
4 8 3
5 2 12

This represents the 5 x 8 matrix whose first row is (0,18,7,6,0,0,0,0)
and whose second row is (0,0,2,1,1,0,0,0), etc..

**The issue**

If we are presented with a list of around 100 million 3-tuples, we'd like a fast way to determine for a given cell (column) the (row,value) pairs for that column.

One strategy is to build a binary file holding the sequence of (row,value) pairs for each column, and creating an *index* that, for each column, tells us the range of the binary file that holds the (row,value) pairs for that column.

For a huge matrix, this will take some time to create, but when we're done, we'll have something resembing a *database* that we can easily and rapidly read from.

In the code below, we use the ctypes package to create a string buffer to load bytes into, and we use the struct package to translate objects (integers in our case) to bytes.

In [65]:
import os
import mylib as my
os.chdir(my.onedrive+"//CurrentCourses//553.688.Fall.2022//Lectures//October//Lecture16")

In [21]:
import struct
import ctypes

#
# read in a small mtx file for starters and load a list of lists of size 3.
# that list is called data
#
# note that mtx format uses row numbering starting from 1 instead of 0.
# we'll convert to starting from 0.
#
fin=open("datas.mtx","r")
data=[]
for line in fin:
    line=line[0:-1]
    fields=line.split(" ")
    if fields[0][0]=="%": # skip comments
        continue
    #print(line)
    # read in the dimensions of the matrix and number of entries
    nrows=int(fields[0])
    ncols=int(fields[1])
    nentries=int(fields[2])
    print(nentries)
    break
for i in range(nentries):
    line=fin.readline()
    line=line[0:-1]
    print(line)
    fields=line.split(" ")
    data.append([int(fields[0])-1,int(fields[1])-1,int(fields[2])])
print(data)

10
1 2 18
1 3 7
1 4 6
2 3 2
2 4 1
2 5 1
4 1 1
4 4 2
4 8 3
5 2 1
[[0, 1, 18], [0, 2, 7], [0, 3, 6], [1, 2, 2], [1, 3, 1], [1, 4, 1], [3, 0, 1], [3, 3, 2], [3, 7, 3], [4, 1, 1]]


**Extract column info**

We now have all of the data as a list of 3 lists, each of length nentries.

Want to store a list consisting of (row,value) pairs for each column. 

We'll keep track separately of  what the range of pairs is for a given column.

In [22]:
#
# Sort the data by column
#
data=sorted(data,key=lambda x:(x[1],x[0]))  
#
# The structure for a pair will consist of 2 unsigned shorts (each is 2 bytes).
# The "H" stands for one unsigned short. In more general situation we can have a structure
# with 1 or more types.
# This is fine if our integers are all less than 256*256-1 = 65,535.
#
sizeforpair=struct.calcsize("!HH")
print("number of bytes for pair = "+str(sizeforpair))
#
# create a buffer to hold all of the data 
#
buffersize=sizeforpair*nentries
print("buffer size = "+str(buffersize))
buffer=ctypes.create_string_buffer(nentries*sizeforpair)
#
# load the data into buffer one entry at a time
# Note:
#
# 1) the ! refers to endianness - when storing a number using multiple bytes
#    which byte comes first
# 2) the HH explains what is stored - in this case two unsigned shorts
# 3) the 3rd argument is an offset - the values get loaded into a position 
#    that is the size of a single pair (4 bytes) times i.
# 3) the final arguments are the 2 integers to be loaded into those 2 positions
#
for i in range(nentries):
    struct.pack_into('!HH',buffer,sizeforpair*i,data[i][0],data[i][2])
#
# write the buffer out to a file
#
with open("datas.bin","wb") as fout:
    fout.write(buffer)

# create columnrange, a list of ncols+1 values
# so that column c appears in locations defined by 
# is range(columnrange[c],columnrange[c+1])
#
colcounts=[0 for i in range(ncols)]
for i in range(nentries):
    col=data[i][1]
    colcounts[col]+=1
cumsum=0
columnrange=[0]
for col in range(ncols):
    cumsum+=colcounts[col]
    columnrange.append(cumsum)
with open("colranges.txt","w") as fout:
    for i in range(ncols+1):
        fout.write(str(columnrange[i])+"\n")

number of bytes for pair = 4
buffer size = 40


In [21]:
columnrange

[0, 1, 3, 5, 8, 9, 9, 9, 10]

**Extracting information from the binary file**

Thus far, we've created a file that we can *query* one column at a time. To make use of this, we need to open the binary file for reading from, and also get the saved list that tells us the record range of each column.

In [24]:
import ctypes
import struct
import time
#
# read in the necessary elements for using this database
#
ncols=5000
colrange=[]
with open("colrange.txt") as fin:
    for i in range(ncols):
        line=fin.readline()
        line=line[0:-1]
        colrange.append(int(line))
sizeforpair=struct.calcsize("!HH")
#
# read the entire database into a byte array "buffer"
#
start_time=time.process_time()
with open("data.bin","rb") as fin:
    buffer=fin.read()
end_time=time.process_time()
print(end_time-start_time)
print("length of buffer = "+str(len(buffer)))

0.0
length of buffer = 1998232


In [26]:
#
# Now use it!!!
#
# get all data for column 47
#
col=47
start_time=time.process_time()
print("length of buffer = "+str(len(buffer)))
for i in range(colrange[col],colrange[col+1]):
    row,value=struct.unpack_from('!HH',buffer,i*sizeforpair)
    print(row,value)
end_time=time.process_time()
print(end_time-start_time)

length of buffer = 1998232
112 1
222 9604
223 81
238 1
303 5929
624 7921
645 289
702 484
705 729
717 7225
782 5625
1337 9216
1388 1600
1443 1369
1504 5041
1513 529
1671 2209
1736 1936
1772 4489
1889 8281
1986 2916
2062 5184
2080 1444
2107 3025
2183 4624
2187 484
2295 4900
2407 25
2604 3844
2724 289
2753 324
2790 2116
2905 324
2988 289
3507 5184
3512 144
3560 1764
3658 1
3759 5776
3848 8464
3855 6724
3864 1936
3879 1369
3903 1
3949 2304
4003 8836
4016 3600
4338 676
4477 196
4643 1521
4717 6084
4776 7396
4833 9
4962 1024
5016 36
5215 1681
5246 1521
5330 7225
5567 6241
5606 8649
5625 961
5770 2704
6150 4225
6196 5329
6247 3600
6310 6241
6335 169
6379 9025
6406 3025
6518 3844
6538 1369
6547 5041
6565 6241
6725 5329
6769 8464
6924 6400
6936 81
6942 25
7193 9801
7239 2916
7288 9025
7444 1156
7545 1089
7608 2025
7612 400
7701 9025
7720 7921
7879 1
7897 3136
7906 49
7933 144
7956 3969
8072 625
8178 4356
8185 2401
8187 4
8298 9801
8454 9
8461 81
8504 6561
8783 3481
8795 2209
8851 484
8931 441
8

Importantly, our data file might be so large that we don't want to read the whole thing in. We can keep it on disk and just extract some bytes as needed.

In [27]:
ncols=5000
colrange=[]
with open("colrange.txt") as fin:
    for i in range(ncols):
        line=fin.readline()
        line=line[0:-1]
        colrange.append(int(line))

sizeforpair=struct.calcsize("!HH")

In [30]:
#
# get data for column 47 witout reading the entire file
#
col=47
fin=open("data.bin","rb")
print("column 3 data")
for i in range(colrange[col],colrange[col+1]):
    fin.seek(i*sizeforpair)
    buffer=fin.read(sizeforpair)
    row,value=struct.unpack_from('!HH',buffer)
    #print(row,value)
fin.close()


column 3 data


## We've seen two important principles behind the creation of efficient databases.

## 1. Storing as binary chunks 
## 2. Use of an index